In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

In [ ]:
# !pip install torch

In [1]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base-openai-detector")

model = AutoModelForSequenceClassification.from_pretrained("roberta-base-openai-detector")

Downloading:   0%|          | 0.00/518 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [20]:
config = AutoConfig.from_pretrained("roberta-base-openai-detector")

In [2]:
import torch

In [3]:
model.to(torch. device('cuda'))

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [102]:
text = "not good"
#text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
input_ids = encoded_input.to('cuda')
output = model(**input_ids)

In [103]:
output

(tensor([[ 3.4616, -2.8279]], device='cuda:0', grad_fn=<AddmmBackward0>),)

In [97]:
# from scipy.special import softmax
# scores = output[0][0].detach().cpu().numpy()
# scores = softmax(scores)

In [100]:
# scores = output[0][0].detach().cpu().numpy()
# #scores = softmax(scores)
# #print(scores)
# predicted_probability = np.max(scores)
# predicted_label = np.argmax(scores)
# #label = "negative" if predicted_probability >= 0.5 
# print(scores)
# print(predicted_probability)
# print(predicted_label)

[ 3.4760382 -2.8418295]
3.4760382
0


In [105]:
scores = output[0][0].detach().cpu().numpy()
scores = softmax(scores)
predicted_probability = np.max(scores)
predicted_label = np.argmax(scores)
threshold = 0.5 # Set the classification threshold

if predicted_probability >= threshold:
    label = "positive"
else:
    label = "negative"

print("Scores:", scores)
print("Predicted probability:", predicted_probability)
print("Predicted label:", predicted_label)
print("Label:", label)

Scores: [0.9981477  0.00185225]
Predicted probability: 0.9981477
Predicted label: 0
Label: positive


In [99]:
scores

array([0.99819946, 0.00180054], dtype=float32)

In [65]:
import numpy as np

In [66]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) LABEL_0 0.9997
2) LABEL_1 0.0003


In [23]:
for i in range(scores.shape[0]):
    print(i)

0
1


In [52]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    label_id = ranking[i]
    if label_id == 0:
        label = "negative"
    elif label_id == 1:
        label = "positive"
    s = scores[label_id]
    print(f"{label} {np.round(float(s), 2)}")

negative 0.88
positive 0.12
